In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [10]:
link_base = 'https://bonbanh.com/oto-cu-da-qua-su-dung'
link_list = []
link_list.append(link_base)
all_info = []


#Link truy cập vào từng trang của website
for i in range(2, 11):
    new_link = link_base + f'/page,{i}'
    link_list.append(new_link)

#Lấy data của từng trang
for j in range(len(link_list)):
    name = []
    year_of_manufacture = []
    price_num = []
    response = requests.get(link_list[j])
    soup = BeautifulSoup(response.text, 'html.parser')
    car_find = soup.find('div', id="search_content")

    # Lấy tên và năm sx
    car_info = car_find.find_all('div', class_="cb2_02")

    car = [car_info[i].text for i in range(len(car_info))]

    for item in range(len(car)):
        split_name = car[item].replace('-', ' ').split()
        name.append(' '.join(split_name[:-1]))
        year_of_manufacture.append(split_name[-1])
    
    # Lấy giá xe
    car_price = car_find.find_all('div', class_="cb3")
    price = [car_price[i].text for i in range(len(car_price))]

    for money in range(len(price)):
        split_price = price[money].split()
        if 'Tỷ' in split_price:
            new_price = int(split_price[split_price.index('Tỷ') - 1]) * 1000 + int(split_price[split_price.index('Tỷ') + 1])
        elif 'Triệu' in split_price:
            new_price = int(split_price[split_price.index('Triệu') - 1])
        price_num.append(new_price)
    
    #Lấy ảnh 
    car_img = car_find.find_all('div', class_="cb5")
    car_img = car_find.find_all('img')
    img = [car_img[i].get('src') for i in range(len(car_img))]

    for i in range(len(name)):
        all_info.append([f'{name[i]}', f'{year_of_manufacture[i]}', f'{price_num[i]}', f'{img[i]}'])
    

# Lấy tên brand
brand_find = soup.find('div', class_="smenu_block")
brand_find = brand_find.find_all('a')
brand = [brand_find[i].text for i in range(len(brand_find))]

df = pd.DataFrame(all_info, columns=['Name', 'Year of manufacture', 'Price', 'Link image'])
df.to_csv('car_data.csv')


# brand, name, year_of_manufacture, price_num, img